In [68]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)
def j(n,m):return L(xjNew(n,m)/x)
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    return polynomialCoefficient(dg,polynomial)
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def quotientFOvrG(f,g) : 
    return (f.maxima_methods().divide(g))[0]
def remainderFOvrG(f,g) : 
    return (f.maxima_methods().divide(g))[1]
def factor(polynomial):
    fp=polynomial.factor()
    return fp
def numberOfFactors(polynomial):
    fp=polynomial.factor()
    return len(fp)
def factorK(polynomial,k):
    fp=factor(h)
    return fp[k-1]
def factorList(polynomial):
    answer=[]
    nf=numberOfFactors(polynomial)
    for k in [1..nf]:
        answer=answer+[factorK(polynomial,k)]
    return answer
def polynomialExponentList(polynomial):
    pel=[]
    fp=polynomial.factor()
    lf=len(fp)
    for n in [0..lf-1]:
        part = fp[n]
        (base,exponent) = part
        pel=pel+[exponent]
    return pel
def polynomialFactorBaseList(polynomial):
    fbl=[]
    fp=polynomial.factor()
    lf=len(fp)
    for n in [0..lf-1]:
        part = fp[n]
        (base,exponent) = part
        fbl=fbl+[base]
    return fbl

In [2]:
data=[]
for m in [3..500]:
    if 10 in divisors(m):
        print(m)
    data=data+[[m,j(120,m)]]
import pickle
wfile = open('/Users/barrybrent/8mar21no6.txt','a')
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500


In [4]:
import pickle
rfile = open('/Users/barrybrent/8mar21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/10mar21no8.txt','w') # interpolating polynomials
enoughfile=[]
for qpower in [0..120]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=x*s[k][1] 
# times x because, to use polynomialCoefficient, I need a polynomial; not a Laurent series
        cf=polynomialCoefficient(qpower,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[qpower-1,rl]] 
    finish = time.time()
    print([qpower,finish-start])
# -1 because I want to record the coefficients of the qpowers of the original Laurent series
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

[0, 3.476809024810791]
[1, 3.5383589267730713]
[2, 3.4991610050201416]
[3, 3.594343900680542]
[4, 3.532088041305542]
[5, 3.5358738899230957]
[6, 3.5763092041015625]
[7, 3.5106911659240723]
[8, 3.568726062774658]
[9, 3.5482139587402344]
[10, 3.553571939468384]
[11, 3.594465970993042]
[12, 3.520862102508545]
[13, 3.532066822052002]
[14, 3.661604166030884]
[15, 3.5511369705200195]
[16, 3.592189073562622]
[17, 3.567767858505249]
[18, 3.59506893157959]
[19, 3.631237030029297]
[20, 3.5660011768341064]
[21, 3.608276844024658]
[22, 3.559959888458252]
[23, 3.584864854812622]
[24, 3.6208600997924805]
[25, 3.5829501152038574]
[26, 3.615128993988037]
[27, 3.6606009006500244]
[28, 3.621027946472168]
[29, 3.6795859336853027]
[30, 3.7123470306396484]
[31, 3.688579797744751]
[32, 3.6764211654663086]
[33, 3.658989191055298]
[34, 3.6817078590393066]
[35, 3.6548259258270264]
[36, 3.682342767715454]
[37, 3.721776008605957]
[38, 3.702333927154541]
[39, 3.7165682315826416]
[40, 3.744623899459839]
[41, 3.689

In [19]:
import pickle
rsfile = open('/Users/barrybrent/10mar21no8.txt','r')
# SageMath file
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)
rtfile = open('/Users/barrybrent/run20nov20no6','r')
# Mathematica file
L=rtfile.readlines()
rtfile.close()
n = 1
f=s[n][1]
print f
g = D(L[n-1])
print(g)

24*x^3 + 32*x
24*x^3 + 32*x


In [18]:
import pickle
rsfile = open('/Users/barrybrent/10mar21no8.txt','r')
# SageMath file
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)
rtfile = open('/Users/barrybrent/run20nov20no6','r')
# Mathematica file
L=rtfile.readlines()
print [len(s),len(L)]

[121, 99]


In [30]:
import pickle
rsfile = open('/Users/barrybrent/10mar21no8.txt','r')
# SageMath file
rs=pickle.load(rsfile)
rsfile.close()
s=stripQuotationMarks(rs)
rtfile = open('/Users/barrybrent/run20nov20no6','r')
# Mathematica file
L=rtfile.readlines()
rtfile.close()
data=[]
for n in [1..98]:
    f=s[n][1]
    g = D(L[n-1])
    data=data+[expand(D(f-g))]
print[data]

# This shows that the SageMath file and the Mathematica file essentially agree.


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [46]:
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..5]:
    print("=====================================================")
    n=s[k][0]
    print("index: ",n)
    f = D(s[k][1])
    print(factor(f))
    pel=polynomialExponentList(f)
    nf=numberOfFactors(f)
    print([nf,pel])
# checking clause 2, we expect that n > 1 => that the exponent list
# at index n should be [1,1,n+1,1] (the polynomialExponentList routine doesn't 
# assign an exponent to the numerical factor.)

('index: ', 0)
(24) * x * (x^2 + 4/3)
[2, [1, 1]]
('index: ', 1)
(276) * x^2 * (x^4 - 8/69*x^2 - 16/23)
[2, [2, 1]]
('index: ', 2)
(2048) * (x - 2) * (x + 2) * x^3 * (x^4 - 8/27*x^2 - 16/27)
[4, [1, 1, 3, 1]]
('index: ', 3)
(11202) * (x - 2) * (x + 2) * x^4 * (x^6 - 38732/5601*x^4 + 11312/5601*x^2 + 6464/1867)
[4, [1, 1, 4, 1]]
('index: ', 4)
(49152) * (x - 2) * (x + 2) * x^5 * (x^8 - 51968/3375*x^6 + 650144/10125*x^4 - 190976/10125*x^2 - 95488/3375)
[4, [1, 1, 5, 1]]


In [100]:
# This checks clause 2 (i): that A_n = (x-2)(x+2)(x^(n+1)) times another polynomial a_n.
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [3..len(s)-1]:
    f = s[k][1]
    pf=poly.factor()
    n=s[k][0]
    divisor=(x^2-4)*x^(n+1)
    rfg=remainderFOvrG(f,divisor)
    data=data+[rfg]
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [51]:
# this cell and the next give another full check of clause 2 (i);
# also, the fact that the final entry in each polynomial exponent 
#  list is a 1 verifies the claim in clause 2 (iii) that a_n ia irreducible.
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    pel=polynomialExponentList(f)
    print([n,pel])

[0, [1, 1]]
[1, [2, 1]]
[2, [1, 1, 3, 1]]
[3, [1, 1, 4, 1]]
[4, [1, 1, 5, 1]]
[5, [1, 1, 6, 1]]
[6, [1, 1, 7, 1]]
[7, [1, 1, 8, 1]]
[8, [1, 1, 9, 1]]
[9, [1, 1, 10, 1]]
[10, [1, 1, 11, 1]]
[11, [1, 1, 12, 1]]
[12, [1, 1, 13, 1]]
[13, [1, 1, 14, 1]]
[14, [1, 1, 15, 1]]
[15, [1, 1, 16, 1]]
[16, [1, 1, 17, 1]]
[17, [1, 1, 18, 1]]
[18, [1, 1, 19, 1]]
[19, [1, 1, 20, 1]]
[20, [1, 1, 21, 1]]
[21, [1, 1, 22, 1]]
[22, [1, 1, 23, 1]]
[23, [1, 1, 24, 1]]
[24, [1, 1, 25, 1]]
[25, [1, 1, 26, 1]]
[26, [1, 1, 27, 1]]
[27, [1, 1, 28, 1]]
[28, [1, 1, 29, 1]]
[29, [1, 1, 30, 1]]
[30, [1, 1, 31, 1]]
[31, [1, 1, 32, 1]]
[32, [1, 1, 33, 1]]
[33, [1, 1, 34, 1]]
[34, [1, 1, 35, 1]]
[35, [1, 1, 36, 1]]
[36, [1, 1, 37, 1]]
[37, [1, 1, 38, 1]]
[38, [1, 1, 39, 1]]
[39, [1, 1, 40, 1]]
[40, [1, 1, 41, 1]]
[41, [1, 1, 42, 1]]
[42, [1, 1, 43, 1]]
[43, [1, 1, 44, 1]]
[44, [1, 1, 45, 1]]
[45, [1, 1, 46, 1]]
[46, [1, 1, 47, 1]]
[47, [1, 1, 48, 1]]
[48, [1, 1, 49, 1]]
[49, [1, 1, 50, 1]]
[50, [1, 1, 51, 1]]
[51, [1, 1,

In [54]:
# more of the above test of clause 2(i)
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    fbl=polynomialFactorBaseList(f)
    print([n,fbl[0],fbl[1]])

[0, x, x^2 + 4/3]
[1, x, x^4 - 8/69*x^2 - 16/23]
[2, x - 2, x + 2]
[3, x - 2, x + 2]
[4, x - 2, x + 2]
[5, x - 2, x + 2]
[6, x - 2, x + 2]
[7, x - 2, x + 2]
[8, x - 2, x + 2]
[9, x - 2, x + 2]
[10, x - 2, x + 2]
[11, x - 2, x + 2]
[12, x - 2, x + 2]
[13, x - 2, x + 2]
[14, x - 2, x + 2]
[15, x - 2, x + 2]
[16, x - 2, x + 2]
[17, x - 2, x + 2]
[18, x - 2, x + 2]
[19, x - 2, x + 2]
[20, x - 2, x + 2]
[21, x - 2, x + 2]
[22, x - 2, x + 2]
[23, x - 2, x + 2]
[24, x - 2, x + 2]
[25, x - 2, x + 2]
[26, x - 2, x + 2]
[27, x - 2, x + 2]
[28, x - 2, x + 2]
[29, x - 2, x + 2]
[30, x - 2, x + 2]
[31, x - 2, x + 2]
[32, x - 2, x + 2]
[33, x - 2, x + 2]
[34, x - 2, x + 2]
[35, x - 2, x + 2]
[36, x - 2, x + 2]
[37, x - 2, x + 2]
[38, x - 2, x + 2]
[39, x - 2, x + 2]
[40, x - 2, x + 2]
[41, x - 2, x + 2]
[42, x - 2, x + 2]
[43, x - 2, x + 2]
[44, x - 2, x + 2]
[45, x - 2, x + 2]
[46, x - 2, x + 2]
[47, x - 2, x + 2]
[48, x - 2, x + 2]
[49, x - 2, x + 2]
[50, x - 2, x + 2]
[51, x - 2, x + 2]
[52, x - 

In [113]:
# check clause 2 (ii):
# McKay Thompson series from table https://oeis.org/A097340/b097340.txt:
mt=[
[-1, 1],
[0, 24],
[1, 276],
[2, 2048],
[3, 11202],
[4, 49152],
[5, 184024],
[6, 614400],
[7, 1881471],
[8, 5373952],
[9, 14478180],
[10, 37122048],
[11, 91231550],
[12, 216072192],
[13, 495248952],
[14, 1102430208],
[15, 2390434947],
[16, 5061476352],
[17, 10487167336],
[18, 21301241856],
[19, 42481784514],
[20, 83300614144],
[21, 160791890304],
[22, 305854488576],
[23, 573872089212],
[24, 1063005978624],
[25,1945403602764],
[26, 3519965179904],
[27, 6300794030460],
[28, 11164248047616],
[29, 19591528119288],
[30, 34065932304384],
[31, 58718797964805],
[32, 100372723007488],
[33, 170215559855424],
[34, 286470013685760],
[35, 478625723149576],
[36, 794110053826560],
[37, 1308745319975256],
[38, 2143055278039040],
[39, 3487563372381816],
[40, 5641848336678912],
[41, 9074553043554568],
[42, 14515166263443456],
[43, 23093778743102262],
[44, 36552977852071936],
[45, 57567784186189368],
[46, 90226777113575424],
[47, 140752796480416011],
[48, 218578429975461888],
[49, 337945040343588276],
[50, 520271697765971968],
[51, 797652526220573580],
[52, 1218002527825723392],
[53, 1852604006634050072],
[54, 2807138079496716288],
[55, 4237760460302936433],
[56, 6374456847628238848],
[57, 9554873766107770560],
[58, 14273181657059143680],
[59, 21250450411204068910],
[60, 31535729115847852032],
[61, 46650835290143061624],
[62, 68797209365301886976],
[63, 101150679669913197462],
[64, 148280443106626633728],
[65, 216743142763626253712],
[66, 315923191441199824896],
[67, 459218611940943755226],
[68, 665710603285072019456],
[69, 962508846974918603904],
[70, 1388038765923851599872],
[71, 1996639069403279491427],
[72, 2864978197116521938944],
[73, 4100990608911708903432],
[74, 5856297079648098807808],
[75, 8343432715970391209502],
[76, 11859696700297921757184],
[77, 16820105145987654631552],
[78, 23802835313046730063872],
[79, 33611779636250175278886],
[80, 47362494062244172660736],
[81, 66600077798590855556532],
[82, 93460562353103053049856],
[83, 130891485964083426534122],
[84, 182952844329494181838848],
[85, 255227018229957765044016],
[86, 355376219286719031156736],
[87, 493899311443420857952845],
[88, 685157678128482627354624],
[89, 948763597225844233250504],
[90, 1311456320500974276980736],
[91, 1809633323386495729057992],
[92, 2492760414984152205361152],
[93, 3427959082742197097793024],
[94, 4706168520874397834575872],
[95, 6450411048962389429976770],
[96, 8826863296640622526464000],
[97, 12059665023346371597383976],
[98, 16450700321708764061517824],
[99, 22405985191605778318221966],
[100, 30470821082605141257437184]];
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..98]:
    n=s[k][0]
    f = D(s[k][1])
    nf=numericalfactor(f)
    data=data+[nf-mt[k][1]]
print[data]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [39]:
# This checks the clainm in clause 2 (iii) that 
# the factors of A_n are monic.
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [2..len(s)-1]:
    f = D(s[k][1])
    n=s[k][0]
    ff=f.factor()
    
    lf = len(ff) - 1
    for nn in [0..lf]:
        
        part = ff[nn]
        
        fpf=factorpairToFactor(part)
       
        
        lc=leadingCoefficient(fpf)
        
        if lc!=1:
            data=data+[[n,nn]]

print("exceptions:")
print(len(data))

exceptions:
0


In [61]:
# test of claim in clause 2(iii) that n >1 => a_n is monic.
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
no=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    fbl=polynomialFactorBaseList(f)
    lnth=len(fbl)
    if lnth !=4:
        no=no+[n]
    lc=leadingCoefficient(fbl[lnth-1])
    data=data+[lc]
print(data)
print("exceptions to length claim:")
print(no)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
exceptions to length claim:
[0, 1]


In [67]:
# another test of the claim in clause 2(iii) that n >1 => a_n is irreducible.
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
yes=[]
no=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    lf=lastfactor(f)
    tf=lf.is_irreducible()
    if tf==False:no=no+[n]
    if tf:yes=yes+[n]
print([len(yes),len(no)])


[120, 0]


In [96]:
# This checks clause 2 (iv): that the degree of a_n is 2n.
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
degrees=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = s[k][1]
    pf=f.factor()
    rfg=quotientFOvrG(f,x^(n+1)*(x^2-4))
    dg=rfg.degree(x)
    degrees=degrees+[dg-2*n]
print(degrees)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [80]:
# initial check of clause 2(v): the monomials in a_n have even exponents.
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
yes=[]
no=[]
for k in [1..7]:
    print("=================================================================================")
    n=s[k][0]
    print(n)
    f = lastfactor(D(s[k][1]))
    print(f)
    print([n,list(f)])


0
x^2 + 4/3
[0, [4/3, 0, 1]]
1
x^4 - 8/69*x^2 - 16/23
[1, [-16/23, 0, -8/69, 0, 1]]
2
x^4 - 8/27*x^2 - 16/27
[2, [-16/27, 0, -8/27, 0, 1]]
3
x^6 - 38732/5601*x^4 + 11312/5601*x^2 + 6464/1867
[3, [6464/1867, 0, 11312/5601, 0, -38732/5601, 0, 1]]
4
x^8 - 51968/3375*x^6 + 650144/10125*x^4 - 190976/10125*x^2 - 95488/3375
[4, [-95488/3375, 0, -190976/10125, 0, 650144/10125, 0, -51968/3375, 0, 1]]
5
x^10 - 15548948/621081*x^8 + 3737957344/16769187*x^6 - 452733568/621081*x^4 + 1217201408/5589729*x^2 + 4868805632/16769187
[5, [4868805632/16769187, 0, 1217201408/5589729, 0, -452733568/621081, 0, 3737957344/16769187, 0, -15548948/621081, 0, 1]]
6
x^12 - 340526584/9646875*x^10 + 14381852336/28940625*x^8 - 32414868736/9646875*x^6 + 18398435584/1929375*x^4 - 16877848576/5788125*x^2 - 33755697152/9646875
[6, [-33755697152/9646875, 0, -16877848576/5788125, 0, 18398435584/1929375, 0, -32414868736/9646875, 0, 14381852336/28940625, 0, -340526584/9646875, 0, 1]]


In [91]:
# full check of clause 2(v): the monomials in a_n have even exponents.
import pickle
rfile = open('/Users/barrybrent/10mar21no8.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

for k in [1..len(s)-1]:
    yes=[]
    no=[]
    n=s[k][0]
    f = lastfactor(D(s[k][1]))
    lf=list(f)
    llf=len(lf)
    for j in [1..llf]:
        cf=lf[j-1]
        if (cf==0)&(2 in divisors(j)):
            yes=yes+[j]
        if (cf!=0)&(2 in divisors(j)):
            no=no+[j]
    print([n,len(yes),len(no)])

[0, 1, 0]
[1, 2, 0]
[2, 2, 0]
[3, 3, 0]
[4, 4, 0]
[5, 5, 0]
[6, 6, 0]
[7, 7, 0]
[8, 8, 0]
[9, 9, 0]
[10, 10, 0]
[11, 11, 0]
[12, 12, 0]
[13, 13, 0]
[14, 14, 0]
[15, 15, 0]
[16, 16, 0]
[17, 17, 0]
[18, 18, 0]
[19, 19, 0]
[20, 20, 0]
[21, 21, 0]
[22, 22, 0]
[23, 23, 0]
[24, 24, 0]
[25, 25, 0]
[26, 26, 0]
[27, 27, 0]
[28, 28, 0]
[29, 29, 0]
[30, 30, 0]
[31, 31, 0]
[32, 32, 0]
[33, 33, 0]
[34, 34, 0]
[35, 35, 0]
[36, 36, 0]
[37, 37, 0]
[38, 38, 0]
[39, 39, 0]
[40, 40, 0]
[41, 41, 0]
[42, 42, 0]
[43, 43, 0]
[44, 44, 0]
[45, 45, 0]
[46, 46, 0]
[47, 47, 0]
[48, 48, 0]
[49, 49, 0]
[50, 50, 0]
[51, 51, 0]
[52, 52, 0]
[53, 53, 0]
[54, 54, 0]
[55, 55, 0]
[56, 56, 0]
[57, 57, 0]
[58, 58, 0]
[59, 59, 0]
[60, 60, 0]
[61, 61, 0]
[62, 62, 0]
[63, 63, 0]
[64, 64, 0]
[65, 65, 0]
[66, 66, 0]
[67, 67, 0]
[68, 68, 0]
[69, 69, 0]
[70, 70, 0]
[71, 71, 0]
[72, 72, 0]
[73, 73, 0]
[74, 74, 0]
[75, 75, 0]
[76, 76, 0]
[77, 77, 0]
[78, 78, 0]
[79, 79, 0]
[80, 80, 0]
[81, 81, 0]
[82, 82, 0]
[83, 83, 0]
[84, 84, 0]
